In [3]:
import numpy as np
import pandas as pd
import pickle



df = pd.read_csv("HR dataset Project.csv")
## Replacing missing values in educatuion with mode of age category
# Step 1: Create a function to fill null values in 'education' based on 'age'
def fill_education_by_age(row):
    age = row['age']

    # Get the mode of education for individuals with the same age
    mode_education = df[(df['age'] == age)]['education'].mode().values

    # If there's a mode value, use it; otherwise, keep the original value
    return mode_education[0] if len(mode_education) > 0 else row['education']

#Apply the function to fill null values in the 'education' column
df['education'] = df.apply(lambda row: fill_education_by_age(row) if pd.isnull(row['education']) else row['education'], axis=1)

## Replacing Null values in previous_year_rating  with 0 as lenght of service is 1 year for employees for them i.e they are new employees
df['previous_year_rating'].fillna(0,inplace=True)

##ordinal encoding education column
from sklearn.preprocessing import OrdinalEncoder


Education = ['Below Secondary',"Bachelor's","Master's & above"]

enc = OrdinalEncoder(categories=[Education])

df[['education']] = enc.fit_transform(df[['education']])





##onehotencoding on gender recruitmentchannel and Department
from numpy import int32
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False,dtype=int32,drop = 'first')
cols = df[['gender','recruitment_channel','department']]
onehot_encoded = ohe.fit_transform(cols)
# Get the feature names
feature_names = ohe.get_feature_names_out(['gender', 'recruitment_channel','department'])
# Create a new DataFrame with the encoded data and feature names
df_encoded = pd.DataFrame(onehot_encoded, columns=feature_names)
# Concatenate the original DataFrame and the encoded DataFrame
df = pd.concat([df, df_encoded], axis=1)
# Droping columns 'gender','recruitment_channel','department'
df.drop(['gender','recruitment_channel','department'],axis=1,inplace=True)

## Encoding region with the region number
### extract number from region name

df['region'] = df['region'].str.extract('(\d+)')

## checkinf data type of column
df['region'].dtype
df['region'] = df['region'].astype(int)





# creating a total score column
df['total_score'] = df['avg_training_score'] * df['no_of_trainings']

Df = df.copy()

##Feature Scaling
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
cols = df[['avg_training_score','total_score']]
scaled = scalar.fit_transform(cols)
## Droping unscaled columns
df.drop(['avg_training_score','total_score'],axis =1,inplace = True)
# Create a new DataFrame with the encoded data and feature names
scaled_col = pd.DataFrame(scaled, columns=['avg_training_score','total_score'])
# Concatenate the original DataFrame and the encoded DataFrame
df = pd.concat([df, scaled_col], axis=1)

#Feature Reduction
df.drop('employee_id',axis = 1 ,inplace = True)

#Training and test Splt

x = df.drop(['is_promoted'],axis = 1) ## independent variable
y = df['is_promoted']

from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.25,random_state= 0)


#Balancing
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
x_train_sm,y_train_sm = sm.fit_resample(x_train,y_train)
from xgboost import XGBClassifier

# Create an XGBoost classifier with specified parameters

model = XGBClassifier(booster='gbtree',eval_metric='auc',learning_rate=0.01,max_depth=220,min_child_weight=1,n_estimators=581,objective='binary:logistic',scale_pos_weight=1,tree_method='hist')

# Fit the model to your training data
model.fit(x_train_sm.values, y_train_sm.values)









XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=220, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=581, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [41]:
import pickle
pickle.dump(model,open('xgmodelip.pkl','wb') )
pickle.dump(scalar,open('scaleip.pkl','wb') )
pickle.dump(ohe,open('oheip.pkl','wb') )

## Creating Pickel import pickle

In [42]:
model = pickle.load(open('xgmodelip.pkl', 'rb'))
ohe = pickle.load(open('oheip.pkl', 'rb'))
scale = pickle.load(open('scaleip.pkl', 'rb'))

In [43]:
age = 33
education = "Bachelor's"
gender = 'm'
department = 'Sales & Marketing'
region = 28
recruitment = 'sourcing'
training = 1
rating = 5
s_len = 6
score = 51
kpi = 1
award = 0


In [44]:

total = score*training
total 

51

In [45]:
data = [gender, recruitment, department]
encoded = ohe.transform(np.array(data).reshape(1, -1))

C:\Users\shree\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [2]:
unscaled = [score,total]

scaled_score = scale.transform(np.array(unscaled).reshape(1, -1)) 
scaled_score

NameError: name 'score' is not defined

In [47]:

feature = np.concatenate([np.array([ region, education,training,age, rating, s_len, kpi, award]),encoded.flatten(),scale.transform(np.array(unscaled).reshape(1, -1)).flatten()])

C:\Users\shree\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [48]:
feature

array(['28', "Bachelor's", '1', '33', '5', '6', '1', '0', '1', '0', '1',
       '0', '0', '0', '0', '0', '0', '1', '0', '-0.9263588972990832',
       '-0.6556851685614054'], dtype='<U32')

In [49]:
x_train_sm.columns

Index(['region', 'education', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?', 'gender_m',
       'recruitment_channel_referred', 'recruitment_channel_sourcing',
       'department_Finance', 'department_HR', 'department_Legal',
       'department_Operations', 'department_Procurement', 'department_R&D',
       'department_Sales & Marketing', 'department_Technology',
       'avg_training_score', 'total_score'],
      dtype='object')

In [50]:
df.head()

,region,education,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,is_promoted,gender_m,...,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,avg_training_score,total_score
0,7,2.0,1,35,5.0,8,1,0,0,0,...,0,0,0,0,0,0,1,0,-1.075931,-0.701265
1,22,1.0,1,30,5.0,4,0,0,0,1,...,0,0,0,1,0,0,0,0,-0.253282,-0.450574
2,19,1.0,1,34,3.0,7,0,0,0,1,...,0,0,0,0,0,0,1,0,-1.001145,-0.678475
3,23,1.0,2,39,1.0,10,0,0,0,1,...,0,0,0,0,0,0,1,0,-1.001145,0.461029
4,26,1.0,1,45,3.0,2,0,0,0,1,...,0,0,0,0,0,0,0,1,0.718939,-0.154303


In [51]:
Df.head()

,employee_id,region,education,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,...,recruitment_channel_sourcing,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,total_score
0,65438,7,2.0,1,35,5.0,8,1,0,49,...,1,0,0,0,0,0,0,1,0,49
1,65141,22,1.0,1,30,5.0,4,0,0,60,...,0,0,0,0,1,0,0,0,0,60
2,7513,19,1.0,1,34,3.0,7,0,0,50,...,1,0,0,0,0,0,0,1,0,50
3,2542,23,1.0,2,39,1.0,10,0,0,50,...,0,0,0,0,0,0,0,1,0,100
4,48945,26,1.0,1,45,3.0,2,0,0,73,...,0,0,0,0,0,0,0,0,1,73


In [52]:
output = int(model.predict([feature]))

XGBoostError: [12:56:16] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\data\array_interface.h:475: Unicode-3 is not supported.

In [61]:
model = pickle.load(open('xgmodelip.pkl', 'rb'))
ohe = pickle.load(open('oheip.pkl', 'rb'))
scale = pickle.load(open('scaleip.pkl', 'rb'))

age = 33
education = "Bachelor's"
gender = 'm'
department = 'Sales & Marketing'
region = 28
recruitment = 'sourcing'
training = 1
rating = 5
s_len = 6
score = 51
kpi = 1
award = 0

total_score = score * training  # Calculate the total score as score * training
    
data = [gender, recruitment, department]
encoded = ohe.transform(np.array(data).reshape(1, -1))

scaled_score = scale.transform(np.array([score,total_score]).reshape(1, -1))  # Scale the total_score with score

feature = np.concatenate([
np.array([region, education, training, age, rating, s_len, kpi, award]),
encoded.flatten(),
scaled_score.flatten()
])

# Make prediction
output = int(model.predict([feature]))

if output == 1:
    result = "PROMOTED"
else:
    result = "NOT_PROMOTED"



C:\Users\shree\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\shree\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


XGBoostError: [05:35:04] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\data\array_interface.h:475: Unicode-3 is not supported.

In [1]:

import pandas as pd
import pickle



df = pd.read_csv("HR dataset Project.csv")
## Replacing missing values in educatuion with mode of age category
# Step 1: Create a function to fill null values in 'education' based on 'age'
def fill_education_by_age(row):
    age = row['age']

    # Get the mode of education for individuals with the same age
    mode_education = df[(df['age'] == age)]['education'].mode().values

    # If there's a mode value, use it; otherwise, keep the original value
    return mode_education[0] if len(mode_education) > 0 else row['education']

#Apply the function to fill null values in the 'education' column
df['education'] = df.apply(lambda row: fill_education_by_age(row) if pd.isnull(row['education']) else row['education'], axis=1)

## Replacing Null values in previous_year_rating  with 0 as lenght of service is 1 year for employees for them i.e they are new employees
df['previous_year_rating'].fillna(0,inplace=True)

##ordinal encoding education column
from sklearn.preprocessing import OrdinalEncoder


Education = ['Below Secondary',"Bachelor's","Master's & above"]

enc = OrdinalEncoder(categories=[Education])

df[['education']] = enc.fit_transform(df[['education']])





##onehotencoding on gender recruitmentchannel and Department
from numpy import int32
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False,dtype=int32,drop = 'first')
cols = df[['gender','recruitment_channel','department']]
onehot_encoded = ohe.fit_transform(cols)
# Get the feature names
feature_names = ohe.get_feature_names_out(['gender', 'recruitment_channel','department'])
# Create a new DataFrame with the encoded data and feature names
df_encoded = pd.DataFrame(onehot_encoded, columns=feature_names)
# Concatenate the original DataFrame and the encoded DataFrame
df = pd.concat([df, df_encoded], axis=1)
# Droping columns 'gender','recruitment_channel','department'
df.drop(['gender','recruitment_channel','department'],axis=1,inplace=True)

## Encoding region with the region number
### extract number from region name

df['region'] = df['region'].str.extract('(\d+)')

## checkinf data type of column
df['region'].dtype
df['region'] = df['region'].astype(int)




##Feature Scaling
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
cols = df[['avg_training_score']]
scaled = scalar.fit_transform(cols)
## Droping unscaled columns
df.drop(['avg_training_score'],axis =1,inplace = True)
# Create a new DataFrame with the encoded data and feature names
scaled_col = pd.DataFrame(scaled, columns=['avg_training_score'])
# Concatenate the original DataFrame and the encoded DataFrame
df = pd.concat([df, scaled_col], axis=1)

#Feature Reduction
df.drop('employee_id',axis = 1 ,inplace = True)

#Training and test Splt

x = df.drop(['is_promoted'],axis = 1) ## independent variable
y = df['is_promoted']

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.25,random_state= 0)



from xgboost import XGBClassifier

# Create an XGBoost classifier with specified parameters

model = XGBClassifier(booster='gbtree',eval_metric='auc',learning_rate=0.01,max_depth=220,min_child_weight=1,n_estimators=581,objective='binary:logistic',scale_pos_weight=1,tree_method='hist')

# Fit the model to your training data
model.fit(x_train.values, y_train.values)

C:\Users\shree\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=220, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=581, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)